This notebook was used to run a grid search for the optimal batch size and epochs for the final model. 
{'batch_size': 50, 'epochs': 15})

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
import matplotlib.style as style
from keras import layers
from keras import models
from keras import optimizers
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import accuracy_score
from keras.optimizers import SGD
from keras.constraints import maxnorm
style.use('fivethirtyeight')

Using TensorFlow backend.


In [2]:
df_train = pd.read_csv('sign-language-mnist/sign_mnist_train.csv')
df_test = pd.read_csv('sign-language-mnist/sign_mnist_test.csv')

In [3]:
labels = df_train['label'].values

In [4]:
df_train.drop('label', axis=1, inplace=True)

In [5]:
images = df_train.values
images = np.array([np.reshape(i, (28, 28)) for i in images])
images = np.array([i.flatten() for i in images])


In [6]:
label_bin = LabelBinarizer()
labels = label_bin.fit_transform(labels)

In [7]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size = 0.3, random_state = 101)

In [8]:
x_train = x_train / 255
x_test = x_test / 255

In [9]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

In [10]:
def create_model():
    
    activation = 'relu'
    dropout_rate = 0.0
    init_mode = 'uniform'
    weight_constraint=0
    optimizer = 'adam'
    lr = 0.01
    momemntum=0
    
    
    model = Sequential()
    model.add(Conv2D(64, kernel_size=(3,3), activation = 'relu', input_shape=(28, 28 ,1) ))
    model.add(MaxPooling2D(pool_size = (2, 2)))

    model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))

    model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))

    model.add(Flatten())
    model.add(Dense(128, activation = 'relu'))
    model.add(Dropout(0.20))
    model.add(Dense(24, activation = 'softmax'))
    
    model.compile(loss = keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
    
    return model

In [11]:
model = KerasClassifier(build_fn=create_model, batch_size=130, epochs=10)

In [12]:
activation = ['relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear', 'softmax']
momentum= [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
learn_rate = [0.001, 0.01, 0.2, 0.3]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
weight_constraint = [1, 2, 3, 4, 5]
neurons = [1, 5, 10, 15, 20, 25, 30]
init = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
optimizers = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
epochs = [5, 10, 15, 20]
batch_size= [50, 100, 150, 200]
param_grid = dict(epochs=epochs, batch_size=batch_size)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)


In [13]:
grid_result = grid.fit(x_train, y_train)

/home/matthew/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/home/matthew/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/home/matthew/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/home/matthew/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_

Epoch 1/15
19218/19218 [==============================] - 18s 940us/step - loss: 2.0782 - acc: 0.3359
Epoch 2/15
19218/19218 [==============================] - 15s 769us/step - loss: 0.7037 - acc: 0.7569
Epoch 3/15
19218/19218 [==============================] - 13s 699us/step - loss: 0.3358 - acc: 0.8869
Epoch 4/15
19218/19218 [==============================] - 13s 692us/step - loss: 0.1789 - acc: 0.9417
Epoch 5/15
19218/19218 [==============================] - 13s 695us/step - loss: 0.1100 - acc: 0.9643
Epoch 6/15
19218/19218 [==============================] - 14s 712us/step - loss: 0.0666 - acc: 0.9801
Epoch 7/15
19218/19218 [==============================] - 17s 888us/step - loss: 0.0478 - acc: 0.9858
Epoch 8/15
19218/19218 [==============================] - 14s 739us/step - loss: 0.0333 - acc: 0.9896
Epoch 9/15
19218/19218 [==============================] - 14s 741us/step - loss: 0.0287 - acc: 0.9918
Epoch 10/15
19218/19218 [==============================] - 14s 739us/step - loss: 

In [14]:
print((grid_result.best_score_, grid_result.best_params_))

(0.9983869303040728, {'batch_size': 50, 'epochs': 15})


Best batch_size = 50, best_epoch = 15